In [1]:
import pandas as pd
import re

In [2]:
# Specify the file path
file_path = "g_inventor_disambiguated.csv"

# Read the Excel file with two separate sheets
excel_data = pd.read_csv(file_path)

C:\Users\rakti\AppData\Local\Temp\ipykernel_35644\2225034436.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  excel_data = pd.read_csv(file_path)


In [7]:
# Concatenate the first and last names
excel_data["full_name"] = excel_data["disambig_inventor_name_first"] + " " + excel_data["disambig_inventor_name_last"]

# Convert non-string values to strings in the 'full_name' column
excel_data['full_name'] = excel_data['full_name'].astype(str)

In [8]:

# Remove punctuation and spaces from the 'full_name' column
excel_data['clean_name'] = excel_data['full_name'].apply(lambda x: re.sub(r'[^\w\s]', '', x).replace(" ", ""))

In [9]:
excel_data

,Unnamed: 0,patent_id,inventor_sequence,inventor_id,disambig_inventor_name_first,disambig_inventor_name_last,male_flag,attribution_status,location_id,full_name,clean_name
0,0,10000000,0,fl:jo_ln:marron-5,Joseph,Marron,1.0,1,198b0471-16c8-11ed-9b5f-1234bde3cd05,Joseph Marron,JosephMarron
1,1,10000001,1,fl:hy_ln:yu-30,Hyeon Jae,YU,NaN,99,3eb37495-16c8-11ed-9b5f-1234bde3cd05,Hyeon Jae YU,HyeonJaeYU
2,2,10000001,0,fl:su_ln:lee-389,Sun-Woo,Lee,NaN,98,6c4ba08f-16c8-11ed-9b5f-1234bde3cd05,Sun-Woo Lee,SunWooLee
3,3,10000002,2,fl:do_ln:choi-31,Dong-Hyeon,Choi,1.0,1,755a6338-16c8-11ed-9b5f-1234bde3cd05,Dong-Hyeon Choi,DongHyeonChoi
4,4,10000002,3,fl:do_ln:kim-369,Dong Jin,Kim,1.0,1,755a6338-16c8-11ed-9b5f-1234bde3cd05,Dong Jin Kim,DongJinKim
...,...,...,...,...,...,...,...,...,...,...,...
20427561,20427561,T999002,0,2a3616tdt5rl235jmrrfoknyr,Henry C.,Twiggs,1.0,1,ebadff41-16c7-11ed-9b5f-1234bde3cd05,Henry C. Twiggs,HenryCTwiggs
20427562,20427562,T999002,2,fl:da_ln:buck-13,Dan F.,Buck,1.0,1,ebadff41-16c7-11ed-9b5f-1234bde3cd05,Dan F. Buck,DanFBuck
20427563,20427563,T999002,1,fl:ja_ln:suggs-2,James L.,Suggs,1.0,1,f5268030-16c7-11ed-9b5f-1234bde3cd05,James L. Suggs,JamesLSuggs
20427564,20427564,T999003,1,fl:gr_ln:lestina-1,Gregory J.,Lestina,1.0,1,d929a955-16c7-11ed-9b5f-1234bde3cd05,Gregory J. Lestina,GregoryJLestina


In [10]:
# Drop every column except 'clean_name' and 'male_flag'
excel_data_clean = excel_data.drop(columns=[col for col in excel_data.columns if col not in ['clean_name', 'male_flag']])

# Print the updated DataFrame
excel_data_clean

,male_flag,clean_name
0,1.0,JosephMarron
1,NaN,HyeonJaeYU
2,NaN,SunWooLee
3,1.0,DongHyeonChoi
4,1.0,DongJinKim
...,...,...
20427561,1.0,HenryCTwiggs
20427562,1.0,DanFBuck
20427563,1.0,JamesLSuggs
20427564,1.0,GregoryJLestina


In [11]:
# Drop rows with NaN values in 'male_flag' column
excel_data_clean = excel_data_clean.dropna(subset=['male_flag'])

# Print the updated DataFrame
excel_data_clean

,male_flag,clean_name
0,1.0,JosephMarron
3,1.0,DongHyeonChoi
4,1.0,DongJinKim
6,0.0,YunJoKim
7,1.0,CarstenElsasser
...,...,...
20427561,1.0,HenryCTwiggs
20427562,1.0,DanFBuck
20427563,1.0,JamesLSuggs
20427564,1.0,GregoryJLestina


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Assuming df is your DataFrame, 'clean_name' is the feature column and 'gender' is the target column
# We'll assume 'gender' is binary where 1 is male and 0 is female
# Split the data into training and testing sets


X_train, X_test, y_train, y_test = train_test_split(excel_data_clean['clean_name'], excel_data_clean['male_flag'], test_size=0.2, random_state=42)

# Create a pipeline that first creates bag of word vectors then applies logistic regression
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(solver='liblinear')),
])
# Train the model using the training data
pipeline.fit(X_train, y_train)
# Predict the labels of the test data
y_pred = pipeline.predict(X_test)
# Print the performance metrics
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.98      0.61      0.76    408646
         1.0       0.95      1.00      0.97   3171856

    accuracy                           0.95   3580502
   macro avg       0.97      0.81      0.87   3580502
weighted avg       0.96      0.95      0.95   3580502



In [ ]:
import pickle

# Export the pipeline as a pickle file 
with open('lang_pipeline.pkl', 'wb') as file: pickle.dump(pipeline, file)